In [1]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import cv2
import numpy as np
import random
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [2]:
!git clone https://github.com/haianh23102000/hoc_may.git

Cloning into 'hoc_may'...
remote: Enumerating objects: 8328, done.
remote: Counting objects: 100% (8328/8328), done.
remote: Compressing objects: 100% (8325/8325), done.
remote: Total 8328 (delta 2), reused 8328 (delta 2), pack-reused 0
Receiving objects: 100% (8328/8328), 488.02 MiB | 33.93 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Checking out files: 100% (8327/8327), done.


In [3]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data(size):
  transform_train = transforms.Compose([
    transforms.Resize((size,size)),
    transforms.ToTensor()  ])
  transform_test = transforms.Compose([
    transforms.Resize((size,size)), 
    transforms.ToTensor()  ])
  trainset = torchvision.datasets.ImageFolder(root='/content/hoc_may/Data/train', transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root='/content/hoc_may/Data/test', transform=transform_test)
  return TrainTest(train=trainset, test=testset)

In [4]:
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

In [6]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  true_result = []
  pred_result = []

  model.train()
  running_loss = 0.0
  reporting_steps = 18
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    true_result += list(labels.cpu().numpy())
    _, predicted = torch.max(outputs, dim=1)
    pred_result += list(predicted.cpu().numpy())
   
    loss = loss_func(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  return pred_result, true_result

In [7]:
def test_epoch(epoch, model, loader, device):
  true = []
  pred = [] 
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      true += list(labels.cpu().numpy())
      pred += list(predicted.cpu().numpy())
  return pred, true

In [8]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 32
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0

In [9]:
def test_epoch(epoch, model, loader, device):
  true = []
  pred = []
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      true += list(labels.cpu().numpy())
      pred += list(predicted.cpu().numpy())
  return pred, true

In [10]:
def main(model = 'vgg16', size = 32):
  classes = get_classes()
  datasets = prepare_data(size)
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  if model == 'vgg16':
    print("vgg16")
    model = torchvision.models.vgg16()
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=3)
  elif model == 'vgg19':
    print("vgg19")
    model = torchvision.models.vgg19()
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=3)
  elif model == 'resnet50':
    print("resnet50")
    model = torchvision.models.resnet50()
    model.fc = torch.nn.Linear(in_features=2048, out_features=3)

  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    pred, true = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(true, pred, target_names=classes))

In [ ]:
main('vgg16',32)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


vgg16


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0 step 31 ave_loss 1.0980
Epoch 0 step 63 ave_loss 1.0927
Epoch 0 step 95 ave_loss 1.0607
Epoch 0 step 127 ave_loss 0.7948
Epoch 0 step 159 ave_loss 0.6359
Epoch 0 step 191 ave_loss 0.4811
              precision    recall  f1-score   support

          2C       0.57      0.44      0.50       409
          3C       0.50      0.99      0.66       367
          4C       1.00      0.67      0.81       831

    accuracy                           0.69      1607
   macro avg       0.69      0.70      0.65      1607
weighted avg       0.78      0.69      0.69      1607

Epoch 1 step 31 ave_loss 0.3340
Epoch 1 step 63 ave_loss 0.2477
Epoch 1 step 95 ave_loss 0.1329
Epoch 1 step 127 ave_loss 0.2229
Epoch 1 step 159 ave_loss 0.1590
Epoch 1 step 191 ave_loss 0.2097
              precision    recall  f1-score   support

          2C       0.57      0.92      0.70       409
          3C       0.62      0.94      0.75       367
          4C       1.00      0.47      0.64       831

    accurac